# Sentiment Analysis on Movie Reviews: Comparing Vectorization Techniques, Models, and Hyperparameters

## Abstract

Text classification is a critical task in sentiment analysis. The objective of this project is to evaluate the performance of different machine learning models combined with text vectorization techniques to predict sentiment on sentences within the Stanford Sentiment Treebank dataset. We considered four models -- Naive Bayes, Logistic Regression, Support Vector Machines (SVM), and Ridge Classifier; and two vectorization techniques -- Term Frequency-Inverse Document Frequency (TF-IDF) and CountVectorizer. A development dataset was used to calculate the accuracy of each combination. The results revealed that the Ridge Classifier model tied with the Logistic Regression model, when respectively combined with the TF-IDF Vectorizer to achieve the highest accuracy (0.8165). This suggests that these combination are the most suitable for our purposes. Furthermore, the TF-IDF Vectorizer outperformed the CountVectorizer in most cases; this indicates its effectiveness in representing text data. These findings contribute valuable insights to the field of text classification and provide a basis for future research exploring additional models, vectorization techniques, and the impact of feature selection and hyperparameter tuning on model performance. The final report includes a comparison of our best result with the leaderboard for SST-2 available on gluebenchmark.com/leaderboard; unfortunately, we did not place in the top 80. However, our project still provides valuable insight into the use of these techniques. We also reflect on the ethical implications, scientific integrity, potential unintended consequences, and the broader context of machine learning in our solution. The project advises caution regarding issues of bias and privacy, and urges attention to be given to how such projects relate to the social context at large.

## *Keywords: Text classification, Sentiment analysis, Machine learning, Naive Bayes, Logistic Regression, Support Vector Machines, Ridge Classifier, Term Frequency-Inverse Document Frequency (TF-IDF), CountVectorizer, Feature representation, Hyperparameter tuning, Model evaluation*

## Introduction

Sentiment analysis is a sub-field of natural language processing (NLP) that seeks to analyze and interpret the emotions or opinions expressed in textual data. Sentiment analysis has garnered significant attention in recent years because of its wide-ranging applications. By employing various machine learning techniques, sentiment analysis can guide making data-driven decisions and gain insight into public opinion in numerous domains.

In this study, we seek to investigate the effectiveness of different combinations of text vectorization techniques and machine learning classifiers. Specifically, we compare the performance of TfidfVectorizer and CountVectorizer, in conjunction with four classifiers: Multinomial Naive Bayes, Logistic Regression, Support Vector Machines (SVM), and Ridge Classifier. The analysis is performed on a dataset comprising sentences labeled with sentiment scores. The evaluation of the classifier-vectorizer combinations is based on their accuracy in predicting sentiment scores for a development dataset. The study also employs techniques of tuning hyper parameters to optimize the results.

In the end, this study reflects on the ethical implications of sentiment analysis and its potential consequences in real-world applications. It also examines the scientific integrity of the research and suggests improvements to enhance the validity and generalizability of the results. By doing so, the study proffers an understanding of the strengths and limitations of various techniques and classifiers, as well as the broader considerations that must be taken into account when implementing sentiment analysis in practice.

## Methods

As noted above, this study uses the Stanford Sentiment Sentiment Treebank dataset (SST-2), employs both the Term Frequency-Inverse Document Frequency (TF-IDF) and CountVectorizer vectorizers, and uses four machine-learning models: Multinomial Naive Bayes, Logistic Regression, Support Vector Machines (SVM), and Ridge Classifier -- these were chosen based on research conducted on each model's behalf vis-a-vis the task at hand. In addition, it performs a GridSearchCV to optimize the hyperparameters for each model. 


### Libraries and Modules

We begin by employing several libraries and modules to facilitate data processing, feature extraction, and model training and evaluation. Here is a list of the main libraries and modules used, along with a succinct overview of their roles in this project:

1. pandas: This library is used for data manipulation and analysis. It provides data structures like DataFrame, which is employed to store and manage the dataset in a tabular format.

2. numpy: A library for numerical computing in Python. It is used for working with arrays and matrices, performing mathematical operations, and other functions relevant to the numerical aspects of the project.

3. scikit-learn: This library provides a wide range of machine learning algorithms, tools for preprocessing, model evaluation metrics, and utilities for model selection and hyperparameter tuning. In this project, scikit-learn is used for:

    a. Text vectorization: TfidfVectorizer and CountVectorizer
    
    b. Machine learning classifiers: Multinomial Naive Bayes, Logistic Regression, Support Vector Machines (SVM), and Ridge Classifier
    
    c. Model evaluation: accuracy_score and GridSearchCV


4. matplotlib: A plotting library for creating static, animated, and interactive visualizations in Python. In this project, it is used to generate visualizations such as precision-recall curves for each model and vectorizer combination.

5. seaborn: data visualization library based on Matplotlib that provides a high-level interface for creating informative and attractive statistical graphics. I

These libraries and modules are widely used in the data science and machine learning community. They offer a strong foundation for performing the tasks in this project.

In [ ]:
# Import data manipulation and numerical computation libraries
import numpy as np
import pandas as pd

# Import Vectorizers, Machine-Learning Classifiers, and Model Selection libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score, classification_report


# Import visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt

### Dataset

The SST-2 dataset used for this study consists of sentences labeled with sentiment scores. The dataset is divided into two parts: training and development sets. The training set is used to train the machine learning models and the development set is used for tuning hyperparameters and evaluating model performance. In this study, we focus on the training and development sets for model comparison and parameter optimization.

In [ ]:
# Load the datasets
train_data = pd.read_csv('train.tsv', delimiter='\t')
dev_data = pd.read_csv('dev.tsv', delimiter='\t')

#train_data.head()

In [ ]:
#dev_data.head()

### Functions

This project only uses one function that can be applied to comments, while taking into account the specific vectorizer and model.

In [ ]:
# Function to predict the sentiment of a given comment using a specified vectorizer and model
def predict(comment, vectorizer, model):
    features = vectorizer.transform(comment)
    return model.predict(features)

### Class

The purpose of defining a Dataset class and the following code is to organize and manage the input features (X) and labels (y) of a dataset. This makes it easier to handle the data throughout the analysis process. The code works by extracting input features (X_train and X_dev) and labels (y_train and y_dev) from the training and development data, respectively. Then, two Dataset objects are instantiated for the train and dev sets, named train and dev. Finally, the input features and labels are assigned to the corresponding attributes of these Dataset objects.

By using the Dataset class and assigning the data to its attributes, the code streamlines data management throughout the project. This approach provides a clean and organized way to access and manipulate the dataset, making it easier to work with the data in subsequent steps of the analysis, such as preprocessing, feature extraction, model training, and evaluation.

In [ ]:
# Define a Dataset class to store input features (X) and labels (y) of a dataset
class Dataset(object):
    """X, y, and V. The X attribute stores the input features (text sentences), y stores 
    the output labels (sentiment), and V stores the vectorized input text features.
    """
    def __init__(self):
        self.X = None  # Input features
        self.y = None  # Output labels
        self.V = None  # Vectorized input text features (not used in this code snippet)

# Extract input features and labels from the training data
X_train = train_data["sentence"]
y_train = train_data["label"]

# Extract input features and labels from the development data
X_dev = dev_data["sentence"]
y_dev = dev_data["label"]

# Instantiate two Dataset objects for train and dev sets
train = Dataset()
dev = Dataset()

# Assign the input features and labels to the train and dev objects
train.X, train.y = X_train, y_train
dev.X, dev.y = X_dev, y_dev

### Text Vectorizers

Two text vectorization techniques were applied to transform the raw text data into numerical representations suitable for machine learning algorithms. The first technique, TfidfVectorizer, computes the Term Frequency-Inverse Document Frequency (TF-IDF) value for each term in the document, which reflects the importance of a term in a document relative to the entire corpus. The second technique, CountVectorizer, simply calculates the frequency of each term in the document. Both vectorizers were tested with and without the removal of stop words to assess the impact of stop word filtering on model performance. (I removed four other variations of vectorizers because they were less effective and took much longer to complete the required computations.)

In [ ]:
# Define different TfidfVectorizer and CountVectorizer configurations
vectorizers = {
    'TF-IDF': TfidfVectorizer(),
    'CountVectorizer': CountVectorizer(),
}

### Machine-Learning Classifiers

Four machine learning classifiers were selected for this study: Multinomial Naive Bayes, Logistic Regression, Support Vector Machines (SVM), and Ridge Classifier. These classifiers were chosen due to their widespread use in text classification tasks and their varying underlying assumptions and mechanisms, which allows for a comprehensive comparison of their performance in sentiment analysis. I have included SVM because it is a versatile and widely-used machine learning algorithm that is well-suited to sentiment analysis. It is effective in high-dimensional spaces, which are common in text data, and is robust to overfitting. 

In [ ]:
# Define different classifiers
classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': LinearSVC(random_state=42),
    'Ridge Classifier': RidgeClassifier(random_state=42),
}

### Hyperparameters for tuning


Each combination of vectorization technique and classifier is evaluated using 5-fold cross-validation on the training set. A GridSearchCV approach is employed to identify the optimal hyperparameters for each classifier. The best-performing model from each combination was then applied to the development set to predict sentiment scores. Model performance was assessed using accuracy as the primary evaluation metric.

In [ ]:
# Define hyperparameters for GridSearchCV
hyperparameters = {
    'Naive Bayes': {'alpha': [0.1, 0.5, 1.0]},
    'Logistic Regression': {'C': [0.1, 4.281332398719396, 10.0]},
    'SVM': {'C': [0.1, 1.0, 10.0]},
    'Ridge Classifier': {'alpha': [0.1, 1.0, 10.0]},
}

### Results in applying the models

In [ ]:
# Initialize a DataFrame to store the results
results_df = pd.DataFrame(columns=["Vectorizer", "Model", "Accuracy"])

# Toggle the cv_splits
cv_splits = 5

# Increase the number of maximum iterations for the LinearSVC model (This was done to deal with a comment appearing
# after running the code).
classifiers['SVM'] = LinearSVC(random_state=42, max_iter=5000)

# Iterate through each vectorizer and model combination
for vectorizer_key in vectorizers.keys():
    vectorizers[vectorizer_key].fit(train.X)
    train.V = vectorizers[vectorizer_key].transform(train.X)

    for model_key in classifiers.keys():
        # Perform GridSearchCV to find the best hyperparameters for each model
        grid_search = GridSearchCV(classifiers[model_key], hyperparameters[model_key], cv=cv_splits)
        grid_search.fit(train.V, train.y)
        best_params = grid_search.best_params_

        # Fit the model with the best hyperparameters to the vectorized training data
        best_model = classifiers[model_key].set_params(**best_params)
        best_model.fit(train.V, train.y)

        # Initialize a list to store the predictions for each sentence in the dev dataset
        dev_preds = []

        # Loop through each sentence in the dev dataset and make predictions
        for sentence in dev.X:
            prediction = predict([sentence], vectorizers[vectorizer_key], best_model)
            dev_preds.append(prediction[0])

        # Calculate the accuracy
        accuracy = accuracy_score(dev.y, dev_preds)
        
        # Print the results
        print(f"Vectorizer: {vectorizer_key} | Model: {model_key} | Accuracy: {accuracy:.4f}")
        
        # Print the classification report
        print(classification_report(dev.y, dev_preds))
        
        # Append the results to the results DataFrame
        results_df = pd.concat([results_df, pd.DataFrame({"Vectorizer": [vectorizer_key],
                                                  "Model": [model_key],
                                                  "Accuracy": [accuracy]})], ignore_index=True)


## Results

### The numbers

We see above the classification reports for each vectorizer and model; this provides a comprehensive summary of the performance of a classification model. Specifically, it details key evaluation metrics for each class within a multi-class problem. It includes precision, recall, F1-score, and support for each class, as well as macro, micro, and weighted averages of these metrics. Precision measures the proportion of true positives among the predicted positives, while recall calculates the proportion of true positives among the actual positives. The F1-score is the harmonic mean of precision and recall, offering a balanced evaluation of the model's performance. Support represents the number of occurrences of each class in the dataset.

In this study, we evaluate the performance of various machine learning models combined with different text vectorization techniques for text classification. As noted above, the models considered include Naive Bayes, Logistic Regression, Support Vector Machines (SVM), and Ridge Classifier, while the vectorization techniques include Term Frequency-Inverse Document Frequency (TF-IDF) and CountVectorizer. The accuracy of each combination was calculated using a development dataset. The results are as follows:

TF-IDF Vectorizer with Naive Bayes: 0.7856

TF-IDF Vectorizer with Logistic Regression: 0.8165

TF-IDF Vectorizer with SVM: 0.8154

TF-IDF Vectorizer with Ridge Classifier: 0.8165

CountVectorizer with Naive Bayes: 0.7913

CountVectorizer with Logistic Regression: 0.8062

CountVectorizer with SVM: 0.8142

CountVectorizer with Ridge Classifier: 0.7764

The results indicate that the combination of TF-IDF Vectorizer and Ridge Classifier achieved the highest accuracy (0.8165), tying with TF-IDF and Logistic Regression, among all tested combinations. This suggests that the Ridge Classifier model (and the Logistic Regression model), when paired with the TF-IDF Vectorizer, is the most suitable for the given text classification task. The TF-IDF Vectorizer appears to provide a better representation of the text data when compared to the CountVectorizer, as three of the four models tested (Logistic Regression, SVM, and Ridge Classifier) achieved higher accuracies with the TF-IDF Vectorizer. This is likely because the TF-IDF Vectorizer takes into account the relative importance of each term in the document and across the entire corpus.

The Support Vector Machines (SVM) model also performed well, with similar accuracies when combined with both the TF-IDF Vectorizer (0.8154) and CountVectorizer (0.8142). This suggests that the SVM model is relatively robust and less sensitive to the choice of text vectorization technique in this case. Naive Bayes and Logistic Regression models achieved lower accuracies, indicating that they may be less effective for the given text classification task.

### Visualization of results

To aid in the analysis, it is helpful to view the following visualizations. Here is the accuracy comparing the eight combinations of vectorizers and classifiers:

#### Heatmap

In [ ]:
# Pivot the results DataFrame to create a matrix for the heatmap
results_pivot = results_df.pivot_table(index="Vectorizer", columns="Model", values="Accuracy")

# Visualization 1. Heatmap using seaborn
plt.figure(figsize=(12, 6))
sns.set(font_scale=1.2)
heatmap = sns.heatmap(results_pivot, annot=True, fmt=".4f", cmap="YlGnBu", cbar_kws={'label': 'Accuracy'})
plt.title("Accuracy of Different Vectorizer and Model Combinations")
plt.show()


#### Barplot

In [ ]:
# Sort results_df by accuracy in descending order
sorted_results_df = results_df.sort_values(by="Accuracy", ascending=False)

# Create an aesthetically-pleasing custom color palette
custom_palette = sns.color_palette("Reds_r", len(vectorizers.keys()))

# Map the sorted results_df to the custom color palette
color_mapping = {vectorizer: custom_palette[i] for i, vectorizer in enumerate(sorted_results_df["Vectorizer"].unique())}

# Visualization 2. Bar plot comparing the performance of different models and vectorizers
plt.figure(figsize=(12, 6))

sns.barplot(x="Model", y="Accuracy", hue="Vectorizer", data=sorted_results_df, palette=color_mapping)
plt.title("Accuracy of Different Vectorizer and Model Combinations")

# Move the legend below the graph with more separation
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(vectorizers.keys()), fontsize=10)

# Change y-axis plot values to show results better
plt.ylim(0.5, 1.0)

plt.show()

#### Confusion matrices for each vectorizer-classifier combination

Confusion matrix visualizations offer a valuable tool for understanding and improving classification models in this project. These matrices provide the following: an interpretable representation of model performance; highlight misclassifications and imbalanced classes; and enable comparison between different models or configurations. By analyzing confusion matrices, we can easily identify potential weaknesses. In addition, we can make informed decisions on model selection, and guide further improvements through adjustments to feature engineering, class weighting, or other model aspects.

In [ ]:
# Visualization 3

fig, axes = plt.subplots(len(vectorizers.keys()), len(classifiers.keys()), figsize=(12, 12))

for i, vectorizer_key in enumerate(vectorizers.keys()):
    vectorizers[vectorizer_key].fit(train.X)
    train.V = vectorizers[vectorizer_key].transform(train.X)

    for j, model_key in enumerate(classifiers.keys()):
        # Get the best hyperparameters for the current model and vectorizer combination
        grid_search = GridSearchCV(classifiers[model_key], hyperparameters[model_key], cv=5)
        grid_search.fit(train.V, train.y)
        best_params = grid_search.best_params_

        # Fit the model with the best hyperparameters to the vectorized training data
        best_model = classifiers[model_key].set_params(**best_params)
        best_model.fit(train.V, train.y)

        # Initialize a list to store the predictions for each sentence in the dev dataset
        dev_preds = []

        # Loop through each sentence in the dev dataset and make predictions
        for sentence in dev.X:
            prediction = predict([sentence], vectorizers[vectorizer_key], best_model)
            dev_preds.append(prediction[0])

        cm = confusion_matrix(dev.y, dev_preds)
        sns.heatmap(cm, annot=True, fmt="d", ax=axes[i, j], cmap="YlGnBu", cbar=False)
        axes[i, j].set_title(f"{vectorizer_key} | {model_key}", fontsize=12)
        
        # Rotate x-axis and y-axis labels for better visibility
        axes[i, j].set_xticklabels(axes[i, j].get_xticklabels(), rotation=0, ha='right', fontsize=10)
        axes[i, j].set_yticklabels(axes[i, j].get_yticklabels(), rotation=0, fontsize=10)
        axes[i, j].set_aspect('equal')  # Set the aspect ratio of the subplot to 'equal'

# Add some space between subplots
fig.subplots_adjust(hspace=0.5, wspace=0.5)

fig.suptitle("Confusion Matrices for Different Vectorizer and Model Combinations", fontsize=16)
plt.show()


## Discussion

### Scientific Integrity of Project and Potential Unintended Consequences of Analysis 

The results in this study rely on a few key aspects. First, the choice of different vectorizer configurations allows for the exploration of various feature extraction approaches. These can help identify the most suitable method for the given dataset. By comparing the results of different configurations, the study aims to find the optimal combination of parameters for the TfidfVectorizer. Additionally, the use of a train-test split helps to assuage the risk of overfitting and allows for a more accurate assessment of the model's generalization capabilities. This ensures that the model's performance on unseen data is not overly optimistic; this provides more reliable and trustworthy results.

However, if the findings of this study are wrong, it could lead to several consequences. A misinformed choice of feature extraction method may result in a model with lower accuracy than what could have been achieved. This would ultimately affect the quality of the analysis or predictions made by the model. Furthermore, if the model is employed in a real-world application, inaccurate predictions could lead to ill-advised decision making, and this could cause significant issues. Therefore, it is crucial to verify the findings of the study, by possibly considering additional performance metrics, performing more cross-validation, or conducting further research using alternative machine learning models or feature extraction techniques. This will help ensure that the conclusions drawn are accurate.

### Ethical Discussion:

#### Potential for bias

The development and deployment of machine learning models for sentiment analysis, such as the one created in this project, raises a number of ethical considerations. One key concern is the potential for biases within the training dataset. Sentiment analysis models rely heavily on the quality and representativeness of the data they are trained on. If the dataset contains biased or unbalanced data, the model could be prone to perpetuating these biases in its predictions. This could lead to unfair treatment of certain groups or reinforce stereotypes, which might be particularly concerning when analyzing user-generated content like movie reviews. To mitigate this, it is crucial to ensure that the training data is carefully curated and diverse, and represents a wide range of perspectives.

Another ethical consideration is the potential for bias in the sentiment analysis models. Machine learning algorithms are trained on text data, which may contain inherent biases that reflect societal prejudices and stereotypes. These biases can be propagated and even amplified by the models, leading to unfair treatment of certain individuals or groups. To address this issue, researchers and practitioners must be carefully attentive to the development of fair and disinterested sentiment analysis models, which requires a continuous effort to evaluate and improve their performance across various demographics and contexts. By actively acknowledging and addressing these ethical concerns, researchers can contribute to the development of more responsible artificial intelligence systems.

#### Accuracy and the social context

Another concern is the accuracy and reliability of the machine learning models used in the classification process. Incorrect classification of sentiments may lead to misinterpretation of public opinion, potentially impacting decision-making processes in business, politics, and other domains. Additionally, these inaccuracies may have unintended consequences on individual users or communities, particularly when sentiment analysis is used to guide the creation and distribution of content on, say, social media platforms. To mitigate these risks, it is crucial to improve the accuracy of sentiment analysis models, and to be transparent about their limitations when reporting results.

#### Issues related to privacy

The application of sentiment analysis models in real-world scenarios also raises concerns related to privacy and surveillance. Organizations might use these models to analyze and monitor public opinions on various platforms, potentially infringing on individuals' right to privacy. Additionally, the automated nature of sentiment analysis might lead to misinterpretations or misclassifications, which could have adverse consequences for individuals or businesses. For instance, a false negative or positive sentiment classification could impact a person's online reputation or a company's brand image. It is pivotal for developers and organizations employing these models to be forthright about their use and to establish guidelines for addressing potential errors and ensuring the responsible handling of sensitive information. Moreover, the broader implications of machine learning models should be considered, including the potential for exacerbating existing power imbalances and inequalities in society.


#### Human and social context of machine learning more broadly

In the context of this project, machine learning has a significant impact on human and social interactions. By automating the process of understanding and quantifying the sentiment expressed in a piece of text, it can help organizations or individuals better comprehend their audience's opinions and emotions. This, in turn, can lead to more informed decision making; it can also result in more improved products and services, and more effective communication strategies. 


### Comparison against SST-2 leadership board

In this project, the best result achieved is an accuracy of 0.8165 using the combination of TF-IDF vectorization and Ridge Classifier. It is also important to note that the SST-2 leaderboard features more advanced models, typically based on deep learning techniques, such as BERT and RoBERTa. These models have reported significantly higher accuracy scores, often exceeding 0.90 or even 0.95 generally. On the SST-2 dataset, the top 16 are all above 97%.

Given the current state of research in the field, the accuracy of 0.8165 (placing it somewhere outside the top 80) would not be competitive with the top-performing models on the SST-2 leaderboard. However, it is still a valuable contribution as it demonstrates the performance of traditional machine learning models and feature extraction techniques, providing a useful comparison and baseline for other researchers.


## Conclusion

This project aimed to investigate the performance of various text classification models and feature extraction techniques in the context of sentiment analysis. We compared four classification models (Naive Bayes, Logistic Regression, Support Vector Machine, and Ridge Classifier) using two feature extraction methods (TF-IDF and CountVectorizer). Our results demonstrated that the combination of TF-IDF vectorization with Ridge Classifier achieved the highest accuracy, tying with TF-IDF and Logistic Regression. Through our analysis, we also highlighted the importance of visualizing performance metrics, such as confusion matrices and bar plots, to gain deeper insights into the model's performance. We also provided a table of the classification reports for each vectorizer and model combination.

Furthermore, we reflected on the ethical implications of our study and acknowledged potential biases in the data, as well as the need for transparency in the modeling process. In terms of scientific integrity, we discussed the limitations of our study and provided suggestions for future research, such as using more diverse and larger datasets, experimenting with additional feature extraction techniques, and employing more advanced classification models.

As a final takeaway, this project demonstrates that the choice of both vectorization and classification methods significantly impacts the performance of sentiment analysis tasks. By carefully selecting the most appropriate techniques and visualizing performance metrics, researchers can optimize their models for a specific task and ensure better performance. Our findings also underscore the importance of considering ethical and scientific integrity aspects when conducting research in natural language processing and text classification domains.

## References

Grus, Joel. (2019). Data Science from Scratch, 2nd Edition: First Principles with Python. O'Reilly Media.

Jha, Atul Anand. (n.d.). Stanford Sentiment Treebank v2 (SST2). [Data set]. Kaggle. Retrieved March 29 2023, from https://www.kaggle.com/datasets/atulanandjha/stanford-sentiment-treebank-v2-sst2.

Misheva, V. (2021). Sentiment Analysis in Python. [DataCamp course]. DataCamp. https://www.datacamp.com/courses/sentiment-analysis-in-python

O'Connor, Brendan. "Introduction to Computational Thinking and Data Science." MIT OpenCourseWare, Massachusetts Institute of Technology, Fall 2020, https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-0002-introduction-to-computational-thinking-and-data-science-fall-2020/index.htm. Accessed on 10 April 2023.

Socher, R., Perelygin, A., Wu, J., Chuang, J., Manning, C. D., Ng, A. and Potts, C. 2013. Recursive deep
models for semantic compositionality over a sentiment treebank. In Proceedings of EMNLP, pp. 1631–1642.

VanderPlas, J. (2016). Python Data Science Handbook: Essential Tools for Working with Data. O'Reilly Media, Inc.

w3schools. "Python Machine Learning". https://www.w3schools.com/python/python_ml_cross_validation.asp. Accessed on 10 April 2023.
